In [1]:
import numpy as np
import pandas as pd
from pycaret.classification import *
import datetime
import matplotlib.pyplot as plt

dataset_s = pd.read_csv('./RB99_1m_X_0.2_14596_1448_Label_513/RB99_1m_X_0.2_14596_1448.csv_tz80_Train_13118.csv')   ############# 训练集文件 ####################
dataset = dataset_s
num_xunlian = len(dataset_s)

# dataset.replace([np.inf, -np.inf], np.nan, inplace=True)   ####替换正负inf为NA

cf = 2 #### 二分类是2，三分类是3

data_1_size = 1448     ###### 测试数据行数  ###############
m_size = 25     ####### 测试多少个月 #######
buy = 1     ##### 多 ###################
sell = 0     ##### 空 ####################
rrr = 0.25     ###### 系数 ###################
m = 1000     ###### 总资金 ###################

In [2]:
res1 = []
res2 = []
res3 = []
res4 = []
res5 = []
res6 = []
res7 = []
resP = []
resR = []
resF = []

for j in range(1,41):        ########## 400个特征 #############
    num = j
    s = setup(dataset, target = 'A0', session_id = 369, pca = True, pca_components = num)    
    
    abc = create_model('xgboost')  ################  xgboost,lightgbm,catboost #############
    
    # abc = create_model('xgboost', objective='binary:logitraw')  ################  不同的目标函数  #############
    
    # abc = create_model('lightgbm', objective='xentlambda')  ################  不同的目标函数  #############
    
    # abc = create_model('catboost', objective='CrossEntropy')   ################  不同的目标函数  #############

    
    abc_results = pull()
    abc_results = abc_results.loc[['Mean']]
    abc_results.to_csv('./temp/'+str(j)+'r.csv',index = False)
    
    final_best = finalize_model(abc)
    save_model(final_best, './temp/' + str(num) + 'x')
    data = pd.read_csv('./RB99_1m_X_0.2_14596_1448_Label_513/RB99_1m_X_0.2_14596_1448.csv_tz80_Test_1448_PCA.csv') #########  测试集文件  ########################
    
#     data.replace([np.inf, -np.inf], np.nan, inplace=True)   ####替换正负inf为NA
    
    predictions = predict_model(final_best, data=data) 
    
    n_preds = predictions['prediction_label'][num_xunlian:(num_xunlian+data_1_size)]    ### 取中间的数据
    n_preds = n_preds.reset_index(drop=True)                      ### 重置索引
    
    Note=open('./temp/' + str(num) + 'x.txt',mode='a')
    for i in range(0,data_1_size):         
        Note.write(str(n_preds[i]) + '\n') 
    Note.close()

    n_preds_score = predictions['prediction_score'][num_xunlian:(num_xunlian+data_1_size)]      ### 取中间的数据
    n_preds_score = n_preds_score.reset_index(drop=True)                   ### 重置索引
    
    Note=open('./temp/' + str(num) + 's.txt',mode='a')
    for i in range(0,data_1_size):         
        Note.write(str(n_preds_score[i]) + '\n') 
    Note.close()  
    
    file_name ='./temp/Show.csv'
    df = pd.read_csv(file_name)
    path = './temp/'+str(j)+'x.txt'
    df2 = pd.read_csv(path, header=None, names=['state_x'])
    for i in range(0,data_1_size):  
        df['low'][i] = df2['state_x'][i]
   
    path = './temp/'+str(j)+'s.txt'
    df2 = pd.read_csv(path, header=None, names=['state_x'])
    df['score'] = 0
    for i in range(0,data_1_size):  
        df['score'][i] = df2['state_x'][i]
    df.to_csv('./temp/'+str(j)+'x.csv',index = False)
    

    file_name='./temp/'+  str(j) + 'x.csv'
    data_1_new = pd.read_csv(file_name)

    aaa1 = data_1_new['volume']
    bbb1 = data_1_new['low']

    if buy == 0:
        for i in range(0,data_1_size):
            if bbb1.iloc[i] == 1:
                aaa1.iloc[i] = aaa1.iloc[i] * -1
    else:
        for i in range(0,data_1_size):
            if bbb1.iloc[i] == 0:
                aaa1.iloc[i] = aaa1.iloc[i] * -1
                
            if cf == 3 and bbb1.iloc[i] == 2:                #########################  三分类
                aaa1.iloc[i] = 0

    data_1_new['high'] = data_1_new['volume'].cumsum()
    data_1_new['open'] = rrr * data_1_new['high'] / m    
    
    ######################################################################################################

    ### 胜率
    wp_win = data_1_new['volume'] > 0
    wp_lost = data_1_new['volume'] < 0
    wp_nothing = data_1_new['volume'] == 0

    wp_win_a = wp_win.sum()            
    wp_lost_a = wp_lost.sum()
    wp_nothing_a = wp_nothing.sum()

    ### 盈亏比
    rrr_win = data_1_new.loc[wp_win, 'volume'].sum()
    rrr_lost = data_1_new.loc[wp_lost, 'volume'].sum()

    ##############################################################################################
    ###### 计算回撤数据
    data_1_new['cum_max_open'] = data_1_new['open'].cummax()  
    data_1_new['down'] = data_1_new['open'] - data_1_new['cum_max_open']  
    data_1_new['down'] = data_1_new['down'].clip(upper=0)  

    ##############################################################################################
    ###### 计算回撤面积
    downarea = data_1_new['down'].sum()

    ##############################################################################################
    ### 二级模型预留
    

    data_1_new['re'] = data_1_new['close'].pct_change() * 100
    data_1_new['real'] = (data_1_new['close'] >= data_1_new['close'].shift(1)).astype(int)
    data_1_new.loc[0, 'real'] = 0 

    if buy == 0:
        data_1_new['real_lab'] = np.where(
            data_1_new['low'] != data_1_new['real'], 
            'G', 'N'
        )
    else:
        data_1_new['real_lab'] = np.where(
            data_1_new['low'] == data_1_new['real'], 
            'G', 'N'
        )

    data_1_new.loc[0, 'real_lab'] = 'G' 

    file_name = './temp/Show.csv'
    df = pd.read_csv(file_name)        
    data_1_new['show'] = df['low']

    data_1_new['show_lab'] = np.where(
        data_1_new['low'] == data_1_new['show'], 
        'G', 'N'
    )

    data_1_new.loc[0, 'show_lab'] = 'G'  

    ##############################################################################################


    if sell == 0:
        data_1_new['re_real'] = np.where(
            data_1_new['low'] == 0, 
            -data_1_new['re'], 
            data_1_new['re']
        )
    else:
        data_1_new['re_real'] = np.where(
            data_1_new['low'] == 1, 
            -data_1_new['re'], 
            data_1_new['re']
        )

    data_1_new.loc[0, 're_real'] = 0  

    ##############################################################################################

    ###### 计算夏普比率和索提诺比率
    re_real = data_1_new['re_real'][1:]  
    mean_re = re_real.mean()
    std_re = re_real.std()
    sharpe = round(mean_re / std_re * 100 if std_re != 0 else 0, 4)

    neg_re = re_real[re_real < 0]
    std_neg_re = neg_re.std() if not neg_re.empty else 0
    sortino = round(mean_re / std_neg_re * 100 if std_neg_re != 0 else 0, 4)

    ##############################################################################################

    data_1_new.to_csv('./temp/' + str(j) + 'x.csv', index=False)

    ###### 计算最大回撤
    cum_max_open = data_1_new['open'].cummax()  ###### 累计最大值
    drawdown = cum_max_open - data_1_new['open']  ###### 回撤值
    s = np.argmax(drawdown)  ###### 最大回撤结束位置

    ###### 确定最大回撤开始位置
    if s == 0:
        e = 0
    else:
        e = np.argmax(data_1_new['open'].iloc[:s]) 

    maxdrawdown = data_1_new['open'].iloc[e] - data_1_new['open'].iloc[s]  ###### 最大回撤
    drawdown_days = s - e  ###### 回撤持续周期数
    
    
    start_DAY = data_1_new.index[s] ######开始回撤的日期
    end_DAY = data_1_new.index[e] ######结束回撤的日期
    start_net_value = data_1_new[data_1_new.index == start_DAY]['open'].values[0] ######开始回撤的净值
    end_net_value = data_1_new[data_1_new.index == end_DAY]['open'].values[0] ######结束回撤的净值
    fig=plt.figure(figsize=(20,11))  
    plt.plot(data_1_new['eob'], data_1_new['open'])
    plt.plot([start_DAY, end_DAY], [start_net_value, end_net_value], linestyle='--', color='r')

    plt.xticks(range(0,data_1_size,int(data_1_size/m_size))) 
    
    plt.legend(['All:' + str(round(data_1_new['open'].iloc[-1]*100,2)) + '%' +
                '   ' + str(m_size) + 'm'
                '   Year:'+ str(round(data_1_new['open'].iloc[-1]/m_size*100*12,2)) + '%' +
                '   CalmarY:'+ str(round((data_1_new['open'].iloc[-1]/m_size*100*12)/(maxdrawdown*100),2)) +
                '   WP:' + str(round(wp_win_a/(wp_win_a + wp_lost_a)*100,2)) + '%' +
                '   RRR:' + str(round(rrr_win/(rrr_win+abs(rrr_lost))*100,2)) + '%' + ' / ' + str(round(rrr_win/abs(rrr_lost),2)) +
                '   T/N:' + str(wp_win_a + wp_lost_a ) + ' / ' + str(wp_nothing_a) +
                '   Sharpe:' + str(sharpe) +
                '   Sortino:' + str(sortino) +
                '   Accuracy:' + str(abc_results['Accuracy'][0]) +
                '   AUC:' + str(abc_results['AUC'][0]) +
                '   Recall:' + str(abc_results['Recall'][0]) +
                '   Prec:' + str(abc_results['Prec.'][0]) +
                '   F1:' + str(abc_results['F1'][0]) +
                '   Kappa:' + str(abc_results['Kappa'][0]) +
                '   MCC:' + str(abc_results['MCC'][0]),

                'MD:'+ str(round(maxdrawdown*100,2)) + '%' +
                '   DA:'+ str(round(downarea,4)) + '%' +
                '   MDT:' + str(drawdown_days)+
                '   Date:' + str(data_1_new['eob'].iloc[e]) + ' - ' + str(data_1_new['eob'].iloc[s])] ,

                loc='upper left',fontsize = 11)   ########### 默认是10
    
    
    plt.plot(data_1_new['eob'], data_1_new['down'], color='#ec700a')   ### 桔色
    plt.fill_between(data_1_new['eob'], data_1_new['down'], 0, where=(data_1_new['down']<0), facecolor='#FF0000', alpha=0.1)   
    plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))                                           ### 红色 + 透明度
          
    fig.autofmt_xdate()
    plt.grid(1)
    plt.savefig("./temp/" + str(j) + "sy.jpg")
    plt.close()

    fig=plt.figure(figsize=(20,10))  
    plt.plot(data_1_new['eob'], data_1_new['high'])
    plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))     ### 最后一个是间隔
    fig.autofmt_xdate()
    plt.grid(1)
    plt.savefig("./temp/" + str(j) + "p.jpg")
    plt.close()   

    
    ##############################################################################################
    
    
    pp = abc_results['Prec.'][0]
    resP.append({
        'Prec_no': j,
        'max_Prec': pp
    })
    
    rr = abc_results['Recall'][0]
    resR.append({
        'Recall_no': j,
        'max_Recall': rr
    })
    
    ff = abc_results['F1'][0]
    resF.append({
        'F1_no': j,
        'max_F1': ff
    })
    
    
    ##############################################################################################    
        
    max_all = round(data_1_new['open'].iloc[-1]*100,2)
    max_no = j

    res1.append({
        'All_no': max_no,
        'max_All': max_all
    })

    max_CalmarY = round((data_1_new['open'].iloc[-1]/m_size*100*12)/(maxdrawdown*100),2)
    
    res2.append({
        'CalmarY_no': max_no,
        'max_CalmarY': max_CalmarY
    }) 
    
    
    res3.append({
        'Downarea_no': max_no,
        'min_Downarea': downarea
    })
          
        
    max_wp = round(wp_win_a/(wp_win_a + wp_lost_a)*100,2)
    
    res4.append({
        'WP_no': max_no,
        'max_WP': max_wp
    })
    
    
    max_rrr = round(rrr_win/(rrr_win+abs(rrr_lost))*100,2)
    
    res5.append({
        'RRR_no': max_no,
        'max_RRR': max_rrr
    })
    
    
    res6.append({
        'Sharpe_no': max_no,
        'max_Sharpe': sharpe
    })
        
        
    res7.append({
        'Sortino_no': max_no,
        'max_Sortino': sortino
    })
    

   ##############################################################################################

aaaP = pd.DataFrame(resP)
aaaR = pd.DataFrame(resR)
aaaF = pd.DataFrame(resF)

bbbP = aaaP.sort_values(by="max_Prec",ascending=False)
bbbR = aaaR.sort_values(by="max_Recall",ascending=False)
bbbF = aaaF.sort_values(by="max_F1",ascending=False)

bbbP = bbbP.reset_index(drop=True)
bbbR = bbbR.reset_index(drop=True)
bbbF = bbbF.reset_index(drop=True)

bbbP['Recall_no'] = bbbR['Recall_no']
bbbP['max_Recall'] = bbbR['max_Recall']
bbbP['F1_no'] = bbbF['F1_no']
bbbP['max_F1'] = bbbF['max_F1']

bbbP.to_csv("./temp/Best_2.csv",index = False)

   ##############################################################################################

aaa1 = pd.DataFrame(res1)
aaa2 = pd.DataFrame(res2)
aaa3 = pd.DataFrame(res3)
aaa4 = pd.DataFrame(res4)
aaa5 = pd.DataFrame(res5)
aaa6 = pd.DataFrame(res6)
aaa7 = pd.DataFrame(res7)

bbb1 = aaa1.sort_values(by="max_All",ascending=False)       ### 由大到小排序
bbb2 = aaa2.sort_values(by="max_CalmarY",ascending=False)    
bbb3 = aaa3.sort_values(by="min_Downarea",ascending=False)     
bbb4 = aaa4.sort_values(by="max_WP",ascending=False)    
bbb5 = aaa5.sort_values(by="max_RRR",ascending=False)    
bbb6 = aaa6.sort_values(by="max_Sharpe",ascending=False)    
bbb7 = aaa7.sort_values(by="max_Sortino",ascending=False)   

bbb1 = bbb1.reset_index(drop=True)
bbb2 = bbb2.reset_index(drop=True)
bbb3 = bbb3.reset_index(drop=True)
bbb4 = bbb4.reset_index(drop=True)
bbb5 = bbb5.reset_index(drop=True)
bbb6 = bbb6.reset_index(drop=True)
bbb7 = bbb7.reset_index(drop=True)

bbb1['CalmarY_no'] = bbb2['CalmarY_no']
bbb1['max_CalmarY'] = bbb2['max_CalmarY']
bbb1['Downarea_no'] = bbb3['Downarea_no']
bbb1['min_Downarea'] = bbb3['min_Downarea']
bbb1['WP_no'] = bbb4['WP_no']
bbb1['max_WP'] = bbb4['max_WP']
bbb1['RRR_no'] = bbb5['RRR_no']
bbb1['max_RRR'] = bbb5['max_RRR']
bbb1['Sharpe_no'] = bbb6['Sharpe_no']
bbb1['max_Sharpe'] = bbb6['max_Sharpe']
bbb1['Sortino_no'] = bbb7['Sortino_no']
bbb1['max_Sortino'] = bbb7['max_Sortino']

bbb1.to_csv("./temp/Best_1.csv",index = False)


,Description,Value
0,Session id,369
1,Target,A0
2,Target type,Binary
3,Original data shape,"(12773, 401)"
4,Transformed data shape,"(12773, 2)"
5,Transformed train set shape,"(8941, 2)"
6,Transformed test set shape,"(3832, 2)"
7,Numeric features,400
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6034,0.6035,0.3973,0.5269,0.4530,0.1514,0.1551
1,0.6286,0.6415,0.4162,0.5704,0.4813,0.2029,0.2090
2,0.6376,0.6508,0.4092,0.5875,0.4824,0.2171,0.2255
3,0.6387,0.6349,0.3957,0.5935,0.4748,0.2159,0.2262
4,0.6510,0.6677,0.4011,0.6192,0.4868,0.2403,0.2534
5,0.6320,0.6569,0.4336,0.5714,0.4931,0.2127,0.2176
6,0.6107,0.6267,0.3930,0.5390,0.4545,0.1633,0.1683
7,0.6242,0.6266,0.4119,0.5609,0.4750,0.1929,0.1984
8,0.6454,0.6578,0.4065,0.6048,0.4862,0.2311,0.2417


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved


,Description,Value
0,Session id,369
1,Target,A0
2,Target type,Binary
3,Original data shape,"(12773, 401)"
4,Transformed data shape,"(12773, 3)"
5,Transformed train set shape,"(8941, 3)"
6,Transformed test set shape,"(3832, 3)"
7,Numeric features,400
8,Preprocess,True
9,Imputation type,simple


...

Transformation Pipeline and Model Successfully Saved


,Description,Value
0,Session id,369
1,Target,A0
2,Target type,Binary
3,Original data shape,"(12773, 401)"
4,Transformed data shape,"(12773, 41)"
5,Transformed train set shape,"(8941, 41)"
6,Transformed test set shape,"(3832, 41)"
7,Numeric features,400
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7508,0.8251,0.6595,0.7155,0.6864,0.4803,0.4814
1,0.7785,0.8606,0.7108,0.7429,0.7265,0.5406,0.5410
2,0.7685,0.8495,0.6748,0.7411,0.7064,0.5159,0.5175
3,0.7819,0.8428,0.7100,0.7486,0.7288,0.5466,0.5471
4,0.7550,0.8365,0.6504,0.7273,0.6867,0.4866,0.4887
5,0.7606,0.8242,0.6585,0.7341,0.6943,0.4985,0.5005
6,0.7640,0.8315,0.7046,0.7182,0.7114,0.5118,0.5118
7,0.7685,0.8450,0.6640,0.7470,0.7030,0.5144,0.5167
8,0.7897,0.8651,0.7182,0.7593,0.7382,0.5627,0.5633


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved
